In [14]:
import numpy as np
from collections import Counter
import graphviz
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


In [2]:
data = pd.read_csv('Breast_Cancer.csv')
data.head()


,Age,Race,Marital Status,T Stage,N Stage,6th Stage,differentiate,Grade,A Stage,Tumor Size,Estrogen Status,Progesterone Status,Regional Node Examined,Reginol Node Positive,Survival Months,Status
0,68,White,Married,T1,N1,IIA,Poorly differentiated,3,Regional,4,Positive,Positive,24,1,60,Alive
1,50,White,Married,T2,N2,IIIA,Moderately differentiated,2,Regional,35,Positive,Positive,14,5,62,Alive
2,58,White,Divorced,T3,N3,IIIC,Moderately differentiated,2,Regional,63,Positive,Positive,14,7,75,Alive
3,58,White,Married,T1,N1,IIA,Poorly differentiated,3,Regional,18,Positive,Positive,2,1,84,Alive
4,47,White,Married,T2,N1,IIB,Poorly differentiated,3,Regional,41,Positive,Positive,3,1,50,Alive


In [5]:
# convert to integer labels
string_labels = ['Race','Marital Status', 'T Stage','N Stage', '6th Stage', 'differentiate', 'A Stage', 'Estrogen Status', 'Progesterone Status', 'Status']
for label in string_labels:
    # convert to integer labels
    data[label] = pd.Categorical(data[label])
    data[label] = data[label].cat.codes 

In [6]:
data.head()

,Age,Race,Marital Status,T Stage,N Stage,6th Stage,differentiate,Grade,A Stage,Tumor Size,Estrogen Status,Progesterone Status,Regional Node Examined,Reginol Node Positive,Survival Months,Status
0,68,2,1,0,0,0,1,3,1,4,1,1,24,1,60,0
1,50,2,1,1,1,2,0,2,1,35,1,1,14,5,62,0
2,58,2,0,2,2,4,0,2,1,63,1,1,14,7,75,0
3,58,2,1,0,0,0,1,3,1,18,1,1,2,1,84,0
4,47,2,1,1,0,1,1,3,1,41,1,1,3,1,50,0


In [8]:
X = data.iloc[:,1:-1].values
y = data.iloc[:,-1].values

In [10]:

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3219, 14), (805, 14), (3219,), (805,))

In [11]:
class Node:
    def __init__(self,feature=None,thresold=None,left=None,right=None,*,value=None):
        self.feature = feature
        self.thresold = thresold
        self.left = left
        self.right = right
        self.value = value


    def is_leaf_node(self):
        return self.value is not None

class DecisionTree:
    def __init__(self,min_samples_split=2,max_depth=100,n_features=None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_features = n_features
        self.root = None
    
    def fit(self,X,y):
        self.n_features = X.shape[1] if not self.n_features else min(self.n_features,X.shape[1])
        self.root = self._grow_tree(X,y)
        
    def _grow_tree(self,X,y,depth=0):
        n_samples,n_features = X.shape
        n_labels = len(np.unique(y))
        
        # Check the stopping criteria
        if (depth >= self.max_depth or n_labels == 1 or n_samples < self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        feature_index = np.random.choice(n_features,self.n_features,replace=False)
        # find the best split
        best_thresold, best_feature =  self._best_split(X,y,feature_index)

        # create the child nodes
        left_indices,right_indices = self._split(X[:,best_feature],best_thresold)
        left = self._grow_tree(X[left_indices,:],y[left_indices],depth+1)
        right = self._grow_tree(X[right_indices,:],y[right_indices],depth+1)

        return Node(best_feature,best_thresold,left,right)


    def _best_split(self,X,y,feature_index):
        best_gain = -1
        split_index,split_thresold = None,None

        for f_idx in feature_index:
            X_column = X[:,f_idx]
            thresold = np.unique(X_column)

            for thres in thresold:
                # calculate the gain
                gain = self._information_gain(y,X_column,thres)

                if gain > best_gain:
                    best_gain = gain
                    split_index = f_idx
                    split_thresold = thres
                
        return split_thresold,split_index
    
    def _information_gain(self,y,X_column,thresold):
        # parent node entropy
        parent_entropy = self._entropy(y)

        # generate the split
        left_indices,right_indices = self._split(X_column,thresold)

        if len(left_indices) == 0 or len(right_indices) == 0:
            return 0
    
        # calculate the child entropy
        n = len(y)
        n_left,n_right = len(left_indices),len(right_indices)
        entropy_left = self._entropy(y[left_indices])
        entropy_right = self._entropy(y[right_indices])

        # calculate the weighted average of the child entropy
        child_entropy = (n_left/n) * entropy_left + (n_right/n) * entropy_right

        # calculate the information gain
        information_gain = parent_entropy - child_entropy

        return information_gain

    def _split(self,X_column,thresold):
        left_indices = np.argwhere(X_column <= thresold).flatten()
        right_indices = np.argwhere(X_column > thresold).flatten()
        return left_indices,right_indices

    def _entropy(self,y):
        counter = Counter(y)
        entropy = 0
        for label in counter:
            p = counter[label] / len(y)
            entropy += -p * np.log2(p)
        return entropy
    
    def _most_common_label(self,y):
        # return np.bincount(y).argmax()
        counter = Counter(y)
        most_common = counter.most_common(1)[0][0]
        return most_common

    def predict(self,X):
        return np.array([self._traverse_tree(x,self.root) for x in X])
    
    def _traverse_tree(self,x,node):
        if node.is_leaf_node():
            return node.value
        
        if x[node.feature] <= node.thresold:
            return self._traverse_tree(x,node.left)
        return self._traverse_tree(x,node.right)   
         

In [12]:
clf = DecisionTree()
clf.fit(X_train,y_train)


In [13]:
y_pred = clf.predict(X_test)

print(f"Accuracy is {accuracy_score(y_test,y_pred)*100:.2f}%")

Accuracy is 86.21%
